In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as stat
import gc

In [ ]:
def calculate_mode(x):
    try:
        moda=stat.mode(x)
    except:
        moda=np.nan
    return moda
def calculate_count(x):
    try:
        count = len(x)
    except:
        count = np.nan
    return count

In [ ]:
# Leer los csv's
path = '/kaggle/input/dataset-reto'

dfuniverse_test = pd.read_csv(f"{path}/universe_test.csv", encoding='utf-8')
dfuniverse_train = pd.read_csv(f"{path}/universe_train.csv",encoding='utf-8')
dfcustomers = pd.read_csv(f"{path}/customers.csv",  encoding='utf-8')

dfliabilities = pd.read_csv(f"{path}/liabilities.csv", encoding='utf-8')
dfmovements = pd.read_csv(f"{path}/movements.csv", encoding='utf-8')
dfbalances = pd.read_csv(f"{path}/balances.csv",  encoding='utf-8')
dfdigital = pd.read_csv(f"{path}/digital.csv", encoding='utf-8')

In [ ]:
# plt.figure(figsize=(10,4))
# axs = sns.countplot(data=dfuniverse_train, x='attrition')
# plt.title('Distribucion del target')
# plt.show()

In [ ]:
id_period_month = ['ID', 'period', 'month']
id_period = ['ID', 'period']

In [ ]:
#### numero de entidades en el S.S. F.F. al que pertenece el cliente en cada periodo
#### numero de tipos de creditos totales asociado al cliente por periodo
#### estadistica básica de su saldo de credito (balance_amount) por periodo
#### estadistica básica de los dias en default del cliente por periodo 
moda=lambda x: calculate_mode(x)
count = lambda x: calculate_count(x)
moda.__name__='mode'
count.__name__='count'
agg_rcc = {'entity':['nunique'],
           'type':[count],
           'balance_amount': ['sum','mean','std','min','max', 'median'],
           'days_default':['sum','max', 'std','median']
          }
dfbalances_ssff = dfbalances.groupby(id_period_month).agg(agg_rcc)
dfbalances_ssff.columns = [feature+'_'+agg for feature, agg in dfbalances_ssff.columns]   
# dfbalances_ssff.head()

In [ ]:
dfbalances_pivot = dfbalances.groupby(id_period_month+['entity'])\
                                         .agg({'balance_amount':['sum','mean','std','min','max','median'],
#                                                'days_default':['sum','std','max','median'],
                                                })\
                                          .unstack(level = 3, fill_value=np.nan)
dfbalances_pivot.columns = [f'{i[0]}_{i[1]}_{i[2]}'  for i in dfbalances_pivot.columns]
# dfbalances_pivot.head()

In [ ]:
dfbalances_transform = dfbalances_ssff.join(dfbalances_pivot, on=id_period_month)
print(dfbalances.shape, dfbalances_ssff.shape, dfbalances_pivot.shape, dfbalances_transform.shape)
del dfbalances, dfbalances_ssff, dfbalances_pivot
# dfbalances_transform.head()

In [ ]:
dfliabilities = dfliabilities.set_index(id_period_month) 
## product_1 (saldo promedio mensual de cta ahorro)
## product_2 (saldo promedio mensual de otras  ctas ahorro)
dfmovements = dfmovements.set_index(id_period_month) 
## type_1 (compras supermercado)
## type_2 (compra restaurantes) 
## type_3 (compras en salud)
## type_4 (compras en transporte)
dfdigital = dfdigital.set_index(id_period_month)

In [ ]:
df_behavior = dfbalances_transform.join(dfliabilities, on=id_period_month, how='outer')\
                                  .join(dfmovements, on = id_period_month,  how='outer')\
                                  .join(dfdigital, on = id_period_month,  how='outer')
print( dfbalances_transform.shape, dfliabilities.shape, dfmovements.shape, dfdigital.shape, df_behavior.shape)
del dfbalances_transform, dfliabilities, dfmovements, dfdigital
# df_behavior.head()

In [ ]:
df_behavior['saldototalpasivo'] = df_behavior['product_1'] + df_behavior['product_2']
df_behavior['ratio_saldototalpasivo'] = df_behavior['balance_amount_mean']/df_behavior['saldototalpasivo']
df_behavior['ratio_product_1'] = df_behavior['balance_amount_mean']/df_behavior['product_1']
df_behavior['ratio_product_2'] = df_behavior['balance_amount_mean']/df_behavior['product_2']
df_behavior= df_behavior.reset_index()

In [ ]:
import gc
gc.collect()

In [ ]:
# ### los clientes en el test no es encuentran en el train
def get_behavior_agg(df, n_ult_meses):
    df['row_num'] = df.groupby(['ID','period']).month.rank(method="first", ascending=False)
    df = df[df['row_num']<= n_ult_meses].drop('row_num', axis=1)
    df = df.groupby(id_period).agg({i:['mean','sum','std'] for i in df.columns if i not in id_period_month})
    df.columns = [f'{i[0]}_ult{n_ult_meses}meses_{i[1]}'  for i in df.columns]
    return df 

list_df = []
for n in [3,6,12]:
    list_df.append(get_behavior_agg(df_behavior, n))
df_behavior_agg_total = pd.concat(list_df, axis=1)
df_behavior_agg_total = df_behavior_agg_total.reset_index()
del list_df, df_behavior

In [ ]:
# type_job
dfcustomers['tiene_product_1'] = dfcustomers['product_1'].map({'Yes':1, 'No':0})
dfcustomers['tiene_product_2'] = dfcustomers['product_2'].map({'Yes':1, 'No':0})
dfcustomers['tiene_product_3'] = dfcustomers['product_3'].map({'Yes':1, 'No':0})
dfcustomers['tiene_product_4'] = dfcustomers['product_4'].map({'Yes':1, 'No':0})
dfcustomers['tiene_ofert_1'] = dfcustomers['ofert_1'].map({'Yes':1, 'No':0})
dfcustomers['tiene_ofert_2'] = dfcustomers['ofert_2'].map({'Yes':1, 'No':0})
dfcustomers['tiene_ofert_3'] = dfcustomers['ofert_3'].map({'Yes':1, 'No':0})
dfcustomers['bureau_risk'] = dfcustomers['bureau_risk'].map({'category_1': 1,'category_2': 2,'category_3': 3,'category_4': 4,
                                            'category_5': 5,'category_6': 6, 'category_7': 7, 'category_8': 8, 'category_9': 9})
dfcustomers.drop(['product_1', 'product_2', 'product_3', 'product_4','ofert_1','ofert_2','ofert_3'], axis=1, inplace=True)
print(dfcustomers.shape)
# dfcustomers.head()

In [ ]:
# df_universe.attrition.astype(str).value_counts(), df_universe.shape
# df_behavior_agg_total
# (df_behavior_agg_total.columns).intersection(df_universe.columns)

In [ ]:
# df_universe = dfuniverse_train.join(dfuniverse_test, on=id_period, how='outer')
df_universe = pd.concat([dfuniverse_train, dfuniverse_test], axis=0)
df_universe = pd.merge(df_universe, df_behavior_agg_total, on=id_period, how='left')
df_universe = pd.merge(df_universe, dfcustomers, on=['ID'], how='left')
print(dfuniverse_train.shape, dfuniverse_test.shape, df_behavior_agg_total.shape, dfcustomers.shape,df_universe.shape )
del dfuniverse_train, dfuniverse_test, df_behavior_agg_total, dfcustomers
df_universe.head()

### MODELLING

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier

In [ ]:
def evaluate_macroF1_lgb(truth, predictions):  
    # this follows the discussion in https://github.com/Microsoft/LightGBM/issues/1483
    pred_labels = predictions.reshape(len(np.unique(truth)),-1).argmax(axis=0)
    f1 = f1_score(truth, pred_labels, average='macro')
    return ('macroF1', f1, True) 

In [ ]:
df_universe = df_universe.drop('type_job', axis=1).set_index(['ID','period'])
dfuniverso_train = df_universe[~df_universe.attrition.isnull()]
dfuniverso_test = df_universe[df_universe.attrition.isnull()]
del df_universe

In [ ]:
columnas_X=dfuniverso_train.drop(columns="attrition").columns
columnas_y=["attrition"]

In [ ]:
train=dfuniverso_train[columnas_X]
test=dfuniverso_test[columnas_X]
y_train=dfuniverso_train[columnas_y]

In [ ]:
# # Dividimos los datos en conjuntos de entrenamiento y prueba
Xt, Xv, yt, yv = train_test_split(train, y_train, test_size=0.2, random_state=42)

In [ ]:
learner = LGBMClassifier(n_estimators=1000,  max_depth=6, 
                         random_state=42, n_jobs=-1,
                         class_weight='balanced', objective='multiclass', num_class=6)

learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric=evaluate_macroF1_lgb,
            eval_set=[(Xt, yt), (Xv, yv)], verbose=10)
# [650]	training's multi_logloss: 0.260274	training's macroF1: 0.510115	valid_1's multi_logloss: 0.761077	valid_1's macroF1: 0.339226
# [590]	training's multi_logloss: 0.278407	training's macroF1: 0.488146	valid_1's multi_logloss: 0.785707	valid_1's macroF1: 0.337969

In [ ]:
import lightgbm as lgb
lgb.plot_importance(learner, max_num_features=30)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
fi = pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns).sort_values(ascending=False)
fi.head(50).to_frame()
# fi.head(520).sum() 0.9905755812445419  588   0.3395310433266527  -- 516   0.3380
# columnas_X = [i for i in fi.head(520).index if 'row_num' not in i]

In [ ]:
from sklearn.metrics import f1_score
y_pred = learner.predict(Xv)
f1_score(yv, y_pred, average='macro')

In [ ]:
result_prediccion = learner.predict(test[columnas_X])

In [ ]:
test["attrition"]=result_prediccion

In [ ]:
envio_df=test.reset_index()[["ID","period","attrition"]]

In [ ]:
envio_df=envio_df.rename(columns={"attrition":"target"})
envio_df

In [ ]:
envio_df.to_csv("envios_0.3380.csv",index=False)